In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import dvu
from copy import deepcopy
import sys
from numpy import ceil
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR, CACHE_DIR, RESULTS_DIR, cache_ngrams_dir, regions_idxs_dir
from neuro.config import repo_dir, PROCESSED_DIR
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
import sasc.viz
from PIL import Image
from sasc import config
from flatmaps_helper import VOX_COUNTS, load_known_rois, ROI_EXPLANATIONS_S03, FED_DRIVING_EXPLANATIONS_S03, FED_DRIVING_EXPLANATIONS_S02
from neuro.flatmaps_helper import load_flatmaps, load_custom_rois


# main load
normalize_flatmaps = False
# gemv_flatmaps_dict_S02, gemv_flatmaps_dict_S03 = load_flatmaps(
    # normalize_flatmaps)
gemv_flatmaps_dict_S02, gemv_flatmaps_dict_S03, gemv_flatmaps_dict_S02_timecourse, gemv_flatmaps_dict_S03_timecourse = load_flatmaps(
    normalize_flatmaps, load_timecourse=True)

### Select avg-response flatmaps

In [ ]:
# select
avg_defaultdict = defaultdict(list)

for subject in ['S02', 'S03']:
    if subject == 'S03':
        gemv_flatmaps_dict = gemv_flatmaps_dict_S03
    elif subject == 'S02':
        gemv_flatmaps_dict = gemv_flatmaps_dict_S02
    common_keys = list(set(gemv_flatmaps_dict_S02.keys()).intersection(
        set(gemv_flatmaps_dict_S03.keys())))
    gemv_flatmaps_dict = {k: v for k,
                          v in gemv_flatmaps_dict.items() if k in common_keys}
    # gemv_flatmaps_dict = {k: v for k, v in gemv_flatmaps_dict.items(
    # ) if not k[0] in ROI_EXPLANATIONS_S03.values() and not k[0] in ['START', 'END']}

    ##################################################
    # rois_dict is a dictionary with keys as region names and np arrays of voxel masks as values

    # rois_dict_known = load_known_rois(subject)
    # rois_dict_comm = load_custom_rois(subject, suffix_setting='')
    rois_dict_fedorenko = load_custom_rois(
        subject, suffix_setting='_fedorenko')
    # rois_dict_spotlights = load_custom_rois(subject, suffix_setting='_spotlights')
    # | rois_dict_spotlights
    # rois_dict = rois_dict_comm | rois_dict_fedorenko  # | rois_dict_known
    # rois_dict = rois_dict_comm
    rois_dict = rois_dict_fedorenko

    ##################################################
    # compute means
    # avg_defaultdict = defaultdict(list)
    for roi_idx, roi in rois_dict.items():
        for explanation in gemv_flatmaps_dict.keys():
            avg_defaultdict[roi_idx + ' ' + subject].append(
                np.mean(gemv_flatmaps_dict[explanation][roi > 0]))
            # corrs_defaultdict[roi_idx].append(
            # np.corrcoef(roi, gemv_flatmaps_dict[explanation])[0, 1])

In [ ]:
expl_set = ['Numbers', 'Years', 'Measurements']

In [ ]:
common_keys

In [ ]:
maps = [
    gemv_flatmaps_dict_S02[(k, None)] for k in expl_set
]

In [ ]:
# Load a sample subject
subject = "UTS02"  # Change this to your actual subject ID
xfmname = "UTS02_auto"  # Change based on your transformation

# Generate three example maps (replace with real data)
volume_shape = maps[0].shape
for map in maps:
    map = (map - map.min()) / (map.max() - map.min())

# Create a 3-channel RGB volume
rgb_volume = cortex.VolumeRGB(
    *maps, subject=subject, xfmname=xfmname)

# Show the visualization
# cortex.quickshow(rgb_volume, with_colorbar=False, fname_save='example_rgb.pdf')
sasc.viz.quickshow(rgb_volume, with_colorbar=False,
                   fname_save='example_rgb.pdf')
plt.show()

# Show flatmaps across subjects

Note: we sometimes normalize them here, so need to reload flatmaps before re-running analyses above!

In [ ]:
normalize_flatmaps = True
normalize_suffix = '_norm' if normalize_flatmaps else ''
gemv_flatmaps_dict_S02_norm, gemv_flatmaps_dict_S03_norm = load_flatmaps(
    normalize_flatmaps=normalize_flatmaps, load_timecourse=False)
gemv_flatmaps_dict_norm = {
    'S02': gemv_flatmaps_dict_S02_norm,
    'S03': gemv_flatmaps_dict_S03_norm
}

In [ ]:
# read a list of pngs and save them as a grid
for expl_list in [
    ['Years', 'Times', 'Numbers', 'Measurements'],
    ['Relationships', 'Dialogue', 'Introspection'],
    ['Gruesome body imagery', 'Fear and Avoidance',
        'Positive Emotional Reactions', 'Negative Emotional Reactions'],
    ['Clothing and Physical Appearance', 'Colors'],
    ['Sexual and Romantic Interactions',
        'Secretive Or Covert Actions', 'Recognition', 'Professions and Personal Backgrounds']
]:

    pngs = defaultdict(list)
    for subject in ['S02', 'S03']:
        for expl in tqdm(expl_list):
            fname = join('custom_rois_flatmaps' +
                         normalize_suffix, subject, expl + '.png')
            if not os.path.exists(fname):

                sasc.viz.quickshow(
                    gemv_flatmaps_dict_norm[subject][(expl, None)],
                    subject=subject,
                    fname_save=fname,)

            pngs[subject].append(fname)

    # plot as grid
    fig, axes = plt.subplots(
        2, len(expl_list), figsize=(len(expl_list) * 5, 6))
    for i, ax in enumerate(axes.flatten()):
        # ax.axis('off')
        # Remove all x-axis ticks and labels
        ax.xaxis.set_ticks([])
        ax.xaxis.set_ticklabels([])
        ax.xaxis.label.set_visible(False)

        # Remove all y-axis ticks and labels (but keep the y-axis label)
        ax.yaxis.set_ticks([])
        ax.yaxis.set_ticklabels([])

        for spine in ax.spines.values():
            spine.set_visible(False)

    for i in range(len(expl_list)):
        axes[0, i].imshow(plt.imread(pngs['S02'][i]))
        axes[0, i].set_title(expl_list[i])
        axes[1, i].imshow(plt.imread(pngs['S03'][i]))
        axes[0, 0].set_ylabel('S02')
        axes[1, 0].set_ylabel('S03')

    # add
    plt.savefig(join('custom_rois_flatmaps' + normalize_suffix, 'full', '__'.join(
        expl_list) + '.png'), bbox_inches='tight', dpi=300)
    plt.show()